In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/learn-ML-DL/langchain-qa-app/

/content/drive/MyDrive/learn-ML-DL/langchain-qa-app


In [3]:
!pip install -r requirements.txt

In [4]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [5]:
from pinecone import Pinecone
pc = Pinecone()

pc.list_indexes()

[
    {
        "name": "churchill-speech",
        "metric": "cosine",
        "host": "churchill-speech-36x87wg.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 768,
        "deletion_protection": "disabled",
        "tags": null
    }
]

# Pinecone Indexes

In [6]:
# from pinecone import ServerlessSpec

# # index_name = 'langchain'

# # if index_name not in pc.list_indexes().names():
# #     print(f'Creating Index: {index_name}')
# #     pc.create_index(
# #         name=index_name,
# #         dimension=1536,
# #         metric='cosine',
# #         spec=ServerlessSpec(
# #             cloud='aws',
# #             region='us-east-1'
# #         )
# #     )
# #     print("Index created")
# # else:
# #     print(f'{index_name} Index Already exist')

In [7]:
# # Deleting Index


# index_name = 'langchain'

# if index_name in pc.list_indexes().names():
#     print(f'Deleting index: {index_name}')
#     pc.delete_index(index_name)
#     print('Index Deleted')
# else:
#     print(f'Index: {index_name} does not exist')

In [8]:
# Get index decription

# select index
# index = pc.Index(index_name)
# index.describe_index_stats()

# Vectors

In [9]:
# # insert vector
# import random

# # dummy vectors
# vectors = [[random.random() for _ in range(1536)] for _ in range(5)]
# ids = list("abcde")

# index_name = 'langchain'

# # create index instance
# index = pc.Index(index_name)

# # upsert to the index
# index.upsert(vectors=zip(ids, vectors))



In [10]:
# update vector

# index.upsert(vectors=[('c', [random.random() for _ in range(1536)])])

In [11]:
# fetch vector by id

# index.fetch(ids=['c', 'd'])

In [12]:
# delete vectors by ids

# index.delete(ids=['b', 'c'])

In [13]:
# index.describe_index_stats()

In [14]:
# index.fetch(ids=['x'])

In [15]:
# query

# query_vector = [random.random() for _ in range(1536)]
# query_vector

In [16]:
# query ops

# index.query(
#     vector=query_vector,
#     top_k=3,
#     include_values=False
# )

# Namespaces

In [17]:
# index.upsert(vectors=zip(ids, vectors), namespace='firstnamespace')

In [18]:
# index.fetch(ids=['x'], namespace='firstnamespace')

In [19]:
# index.delete(ids=['b', 'c'], namespace='firstnamespace')
# index.delete(delete_all=True, namespace='firstnamespace')

# Splitting and Embedding Text

In [20]:
# index.describe_index_stats()

In [21]:

# index_name = 'langchain'

# if index_name in pc.list_indexes().names():
#     print(f'Deleting index: {index_name}')
#     pc.delete_index(index_name)
#     print('Index Deleted')
# else:
#     print(f'Index: {index_name} does not exist')

In [22]:
# split text into chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('churchill_speech.txt') as f:
    churchill_speech = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, # max size of chunks you want to split (experimental value)
    chunk_overlap=20,
    length_function=len
)

In [23]:
chunks = text_splitter.create_documents([churchill_speech])
print(chunks[2].page_content)

From the moment that the French defenses at Sedan and on the Meuse were broken at the end of the


In [24]:
print(len(chunks))

300


In [25]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [26]:
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

vector = embeddings.embed_query("My name is Joshua")
print(len(vector), vector[:5])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


768 [0.07186414301395416, 0.06533984839916229, -0.0019175804918631911, -0.03360580652952194, 0.02873637154698372]


In [27]:
first_chunk = embeddings.embed_query(chunks[0].page_content)
first_chunk[:10]

[0.038391608744859695,
 0.024885855615139008,
 0.00048682864871807396,
 0.01173435989767313,
 -0.01568249799311161,
 -0.021985815837979317,
 0.014491490088403225,
 3.26168374158442e-05,
 -0.033055245876312256,
 0.004893346689641476]

## Inserting Embeddings into Pinecone Index

In [28]:
from pinecone import Pinecone
from langchain_community.vectorstores import Pinecone as LangChainPinecone

pc = Pinecone()

pc.list_indexes()

[
    {
        "name": "churchill-speech",
        "metric": "cosine",
        "host": "churchill-speech-36x87wg.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 768,
        "deletion_protection": "disabled",
        "tags": null
    }
]

In [29]:
# # Deleting Index


# index_name = 'churchill-spech'

def delete_index(index_name:str):
    if index_name in pc.list_indexes().names():
        print(f'Deleting index: {index_name}.....\n')
        print('-' * 100)
        pc.delete_index(index_name)
        print('Index Deleted!!!')
    else:
        print(f'Index: {index_name} does not exist')

    return None



In [30]:
# delete_index(index_name=index_name)

In [31]:
from pinecone import ServerlessSpec

def create_index(index_name:str, dim:int):
    if index_name not in pc.list_indexes().names():
        print(f'Creating Index: {index_name}')
        pc.create_index(
            name=index_name,
            dimension=dim,
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )
        print(f"Index '{index_name}' created")
    else:
        print(f'{index_name} Index Already exist')

    return None

In [32]:
index_name = 'churchill-speech'

create_index(index_name, dim=768)

churchill-speech Index Already exist


In [33]:
# import sys
# if "pinecone" in sys.modules:
#     del sys.modules["pinecone"]
#     print("Done")

In [35]:
!pip show pinecone

Name: pinecone
Version: 7.3.0
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: certifi, pinecone-plugin-assistant, pinecone-plugin-interface, python-dateutil, typing-extensions, urllib3
Required-by: 


In [39]:
!pip install langchain-pinecone

In [42]:
# Upload vectors to pinecone using langchain
from pinecone import Pinecone
from langchain_pinecone import Pinecone as LangChainPinecone


index_name = "churchill-speech"

vector_store = LangChainPinecone.from_documents(
    chunks,
    embeddings,
    index_name=index_name
)

In [48]:
vector_store = LangChainPinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

## Similarity search to ask questions about custom document



In [49]:
query = 'Where will the fight take place?'

result = vector_store.similarity_search(query=query)
print(result)

[Document(id='686a94c7-c4e6-40e2-b627-ec26c88f3032', metadata={}, page_content='shall fight on the beaches, we shall fight on the landing grounds, we shall fight in the fields and'), Document(id='87050a1f-8fb4-49e6-bff0-1e8df40a3d01', metadata={}, page_content='front, now on that, fighting'), Document(id='6f50cf89-c009-4201-838b-86f4cfea4700', metadata={}, page_content='end, we shall fight in France, we shall fight on the seas and oceans, we shall fight with growing'), Document(id='a4d6090d-d6aa-47af-80ab-3ff29c3cd902', metadata={}, page_content='will defend to the')]


In [51]:
for r in result:
    print(r.page_content)
    print('-' * 50)

shall fight on the beaches, we shall fight on the landing grounds, we shall fight in the fields and
--------------------------------------------------
front, now on that, fighting
--------------------------------------------------
end, we shall fight in France, we shall fight on the seas and oceans, we shall fight with growing
--------------------------------------------------
will defend to the
--------------------------------------------------


In [52]:
# Give results of Similarity search to AI model

from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI

chat_llm = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=1.1
)

retriever = vector_store.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 3}
)

chain = RetrievalQA.from_chain_type(
    llm=chat_llm,
    chain_type='stuff',
    retriever=retriever
)

In [59]:
query = 'In which places are fights likely to take place?'

chain.invoke(query)

{'query': 'In which places are fights likely to take place?',
 'result': 'Based on the provided text, fights are likely to take place on the beaches, on the landing grounds, and in the fields.'}

In [62]:
question = input('Question about the famous Churchill speech: ')
print('-' * 100)
print('-' * 100)
answer = chain.invoke(question)

print(answer['result'])

Question about the famous Churchill speech: Who was the king of Belgium as at the time this speech was given?
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
The King of Belgium at the time this speech was given was **King Leopold**.
